In [1]:
import os
import argparse
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset,DataLoader

import torchvision.utils as utils
import torchvision.transforms as transforms
import torchvision.models as models


import copy
import glob
import torch.utils.data as udata
import h5py
import cv2
import random
import pywt
import matplotlib.pyplot as plt
from PIL import Image
import math
from skimage.measure.simple_metrics import compare_psnr
import scipy.io as sio
import random
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [2]:
#本代码，直接将风格合成结果输入残差网络，直接以原图和西电图之间的均方差作为损失，得到西电图
opts = argparse.Namespace()
opts.image_channel = 3 #为什么通道是6
opts.batch_size = 3
opts.lr = 1e-3
opts.target_path = './content/l2.png' #目标图 就是西电内容图
use_cuda = torch.cuda.is_available()
if use_cuda:
    device = "cuda:0" 
    dtype = torch.cuda.FloatTensor 
    imsize = 128 
else:
    device = "cpu"
    dtype = torch.FloatTensor
    # desired size of the output image
    imsize = 128  # use small size if no gpu
print(use_cuda)
print(device)

True
cuda:0


In [3]:

loader = transforms.Compose([
    transforms.Resize([imsize, imsize]),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    image = Variable(loader(image))
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0)
    return image

unloader = transforms.ToPILImage()  # reconvert into PIL image
def imshow(tensor, title=None):
    image = tensor.clone().cpu()  # we clone the tensor to not do changes on it
    image = image.view(3, imsize, imsize)  # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated
    
def image_unloader(tensor):
    image = tensor.clone().cpu()  # we clone the tensor to not do changes on it
    image = image.view(3, imsize, imsize)  # remove the fake batch dimension
    image = unloader(image)
    return image

def pil2tensor(pil_img):
    image = Variable(loader(pil_img))
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0)
    return image

In [4]:
#load content 加载内容图
content_img = image_loader(opts.target_path).type(dtype) #.squeeze()

In [5]:
mat = sio.loadmat('l2_1') #加载数据库

In [6]:
x = mat['img'] #图片
x_name = mat['img_name'].squeeze() #图片名称

In [7]:
a = np.asarray(list(range(x.shape[0])))  #应该是数据集数量
size = int(x.shape[0] * 0.8) #size = c*0.8?
p1 = np.asarray(np.random.choice(a=a, size=size, replace=False)) #从中随机抽取百分之八十的数据作为训练集？

In [8]:
p2 = []
for tmp in a:
    if(tmp not in p1):
        p2.append(tmp)
p2 = np.asarray(p2) #剩下的是测试集

In [9]:
p1.sort() #排序？意义在哪
p2.sort()

In [10]:
train_x = x[p1]
train_x_name = x_name[p1]
test_x = x[p2]
test_x_name = x_name[p2]
target = image_loader(opts.target_path).to('cpu').detach().numpy()
targets = [target for i in range(len(train_x))]
targets = np.asarray(targets).squeeze(1)
train_x = torch.from_numpy(train_x).type(dtype)
train_y = torch.from_numpy(targets).type(dtype)
test_x = torch.from_numpy(test_x).type(dtype)
#[5-10]加载数据集，并随机划分数据为0.8的训练和0.2的测试

In [11]:
train_dataset = TensorDataset(train_x,train_y)
train_loader = DataLoader(train_dataset, batch_size=opts.batch_size, shuffle=True)

In [12]:
test_dataset = TensorDataset(test_x)
test_loader = DataLoader(test_dataset, batch_size=opts.batch_size, shuffle=True)

In [13]:
# 3x3 convolution  网络部分 终于要看了
#残差网络
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class DenoiseNet2(nn.Module):
    def __init__(self, block):
        super(DenoiseNet2, self).__init__()
        self.e1 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )# 64,32,32
        self.e2 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )# 128,32,32
        self.l1 = self.make_block(block, 128, 128)
        self.l2 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64,32,32
        self.l3 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64,32,32
        self.l4 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64 + 64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64,32,32
        self.l5 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64 + 64 + 64, 64, 1, 1, 0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 1, 128, 128
        self.l6 = self.make_block(block, 128 + 64, 128 + 64)
        self.l7 = self.make_block(block, 128 + 64, 128 + 64)
        self.l8 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )# 64,32,32
        self.l9 = nn.Sequential(
            nn.ConvTranspose2d(128 + 128, 64, 3, 2 , 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64, 64, 64
        self.l10 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(64, 3, 1, 1, 0),
            nn.Sigmoid()
        )# 1, 64, 64
    
    def make_block(self, block, in_c, out_c):
        layers = []
        layers.append(block(in_c, out_c, 1, False))
        return nn.Sequential(*layers)

    
    def forward(self, x):
        e1 = self.e1(x)
        e2 = self.e2(e1)
        l1 = self.l1(e2)
        l2 = self.l2(l1)
        in_l3 = torch.cat((l1, l2), dim = 1)
        l3 = self.l3(in_l3)
        in_l4 = torch.cat((l1, l2, l3), dim = 1)
        l4 = self.l4(in_l4)
        in_l5 = torch.cat((l1, l2, l3, l4), dim = 1)
        l5 = self.l5(in_l5)
        in_l6 = torch.cat((l1, l5), dim = 1)
        l6 = self.l6(in_l6)
        l7 = self.l7(l6)
        l8 = self.l8(l7)
        in_l9 = torch.cat((e2, l8), dim = 1)
        l9 = self.l9(in_l9)
        l10 = self.l10(l9)
        return l10

In [14]:
net = DenoiseNet2(ResidualBlock).to(device) #不知道为啥一直要.to （device）可以仔细看看，使用训练要求什么，把所有数据.to(device)?
optimizer = optim.Adam(net.parameters(), lr=opts.lr) #Adam优化器
criterion = nn.MSELoss() #损失使用MSE损失

In [15]:
for epoch in range(1):
    train_loss = 0
    for i, data in enumerate(train_loader):
        x,y = data
        x = x.to(device)
        y = y.to(device)
        pred = net(x)
        optimizer.zero_grad()

        loss = criterion(pred, y) #优化网络，使得输入风格合成图，输出能是那啥，内容
        loss.backward()
        optimizer.step()
        

    print('[Epoch %d] loss: %.8f' % (epoch + 1, loss.item()))

[Epoch 1] loss: 0.01215791


In [16]:
# 保存模型
torch.save(net.state_dict(),'extract_l2_1.pth')
# 加载模型
net.load_state_dict(torch.load('extract_l2_1.pth'))

<All keys matched successfully>

In [17]:
source_path = 'style_attack' #资源，保存的是全部风格图
save_path = './extract_l2/train' #训练输入图片 攻击后合成图 从数据集就自己提取
save_res_path = './extract_l2/train_res' #输出结果  全部提取结果 结果,由网络生成
save_source_path = './extract_l2/train_source' #训练合成资源 对应资源从第一行位置获取
dataset = train_x
dataset_name = train_x_name
for i in range(dataset.shape[0]):
    msave_path = os.path.join(save_path, dataset_name[i].strip())
    msave_res_path = os.path.join(save_res_path, dataset_name[i].strip())
    msave_source_path = os.path.join(save_source_path, dataset_name[i].strip())
    x = dataset[i].clone().unsqueeze(0)
    x = x.to(device)
    pred = net(x)
    x = image_unloader(x)
    x.save(msave_path) #攻击后合成图
    pred = image_unloader(pred)
    pred.save(msave_res_path) #提取结果
    source = image_loader(os.path.join(source_path, dataset_name[i].strip()))
    source = image_unloader(source)
    source.save(msave_source_path) #我觉得有点不对劲

In [18]:
#lst = os.listdir('train_source')

In [19]:
#len(lst)

In [20]:
       
source_path = 'style_attack' #资源，保存的是全部风格图
save_path = './extract_l2/test' #训练输入图片 攻击后合成图
save_res_path = './extract_l2/test_res' #输出结果  全部提取结果
save_source_path = './extract_l2/test_source' #训练合成资源
dataset = test_x
dataset_name = test_x_name
for i in range(dataset.shape[0]):
    msave_path = os.path.join(save_path, dataset_name[i].strip())
    msave_res_path = os.path.join(save_res_path, dataset_name[i].strip())
    msave_source_path = os.path.join(save_source_path, dataset_name[i].strip())
    x = dataset[i].clone().unsqueeze(0)
    x = x.to(device)
    pred = net(x)
    x = image_unloader(x)
    x.save(msave_path) #攻击后合成图
    pred = image_unloader(pred)
    pred.save(msave_res_path) #提取结果
    source = image_loader(os.path.join(source_path, dataset_name[i].strip()))
    source = image_unloader(source)
    source.save(msave_source_path) #我觉得有点不对劲
        
        
        

In [21]:
# IMAGE_PATH = './st_test'
# # SAVE_PATH = './res'
# # source_path = 'res1' #资源，保存的是全部风格图
# # save_path = './test1' #训练输入图片 攻击后合成图
# # save_res_path = './test_res1' #输出结果  全部提取结果
# # save_source_path = './test_source1' #训练合成资源

# image_pathes = os.listdir(IMAGE_PATH)
# images = []
# for img_path in image_pathes:
#     if os.path.isdir(img_path):
#         continue
#     tmp = image_loader(os.path.join(IMAGE_PATH,img_path)).detach().to(device)
#     pred = net(tmp)
#     print('_____________________________________')
#     imshow(tmp,"input")
#     imshow(pred,"output")